In [1]:
packages = !pip list
for x in packages:
    if "qiskit" in x:
        print(x)

qiskit                1.1.0
qiskit-aer            0.14.1
qiskit-ibm-provider   0.11.0
qiskit-ibm-runtime    0.24.0


In [2]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
# from qiskit.tools.jupyter import *
from qiskit.visualization import *
# from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options
token = "#"

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum",token=token)

# Invoke a primitive. For more details see https://docs.quantum.ibm.com/run/primitives
# result = Sampler().run(circuits).result()

In [3]:
# Importing Qiskit
from qiskit import *
import random
#other useful packages
import math
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
# Import Qiskit
from qiskit import transpile
from qiskit_aer import Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
# Preparation for encoding
KEY_LENGTH = 500
# KEY_LENGTH = 300 #Doing this as IBM Public QPUs support max only 300 maximum circuits
random.seed(0) # Seed the random number generator. This will be used as our "coin flipper".
# Generating a random string of bits
alice_bits = ""
for i in range(KEY_LENGTH):
    randBit = random.randint(0, 1) # Flip Coin
    alice_bits += str(randBit) # Add randomly chosen bit to the bit string.
print("The bits Alice is going to send are: " + alice_bits[:10] + "...")

The bits Alice is going to send are: 1101111110...


In [4]:
def generate_random_bases(num_of_bases):
    """This function selects a random basis for each bit"""
    bases_string = ""
    for i in range(num_of_bases):

        randBasis = random.randint(0, 1) # Flip Coin
        if randBasis == 0:
            bases_string += "Z"
        else:
            bases_string += "X"
    return bases_string
alice_bases = generate_random_bases(KEY_LENGTH) # Alice randomly chooses a basis for each bit.
print("The bases Alice is going to encode them in are: " + alice_bases[:10] + "...")

The bases Alice is going to encode them in are: ZXXZZZZXZX...


In [5]:
def encode(bits, bases):
    """This function encodes each bit into the given basis."""
    encoded_qubits = []
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1) # Create a quantum circuit for each qubit
        # Possible Cases
        if bit=="0" and basis == "Z":
            encoded_qubits.append(qc) # Do not apply any gates
        elif bit=="1" and basis == "Z":
            qc.x(0) # Apply X Gate
            encoded_qubits.append(qc)
        elif bit=="0" and basis == "X":
            qc.h(0) # Apply H Gate
            encoded_qubits.append(qc)
        elif bit=="1" and basis == "X":
            qc.x(0) # Apply X Gate
            qc.h(0) # Apply H Gate
            encoded_qubits.append(qc)
    return (encoded_qubits)

In [6]:
# Encode Alice's bits
encoded_qubits = encode(alice_bits, alice_bases)
# Print circuits for first 5 qubits.
for i in range(5):
    print(encoded_qubits[i])
print("etc.")

     ┌───┐
  q: ┤ X ├
     └───┘
c: 1/═════
          
     ┌───┐┌───┐
  q: ┤ X ├┤ H ├
     └───┘└───┘
c: 1/══════════
               
     ┌───┐
  q: ┤ H ├
     └───┘
c: 1/═════
          
     ┌───┐
  q: ┤ X ├
     └───┘
c: 1/═════
          
     ┌───┐
  q: ┤ X ├
     └───┘
c: 1/═════
          
etc.


In [7]:
QUANTUM_CHANNEL = encoded_qubits
bob_bases = generate_random_bases(KEY_LENGTH) # Bob randomly chooses a basis for each bit.
print("The bases Bob is going to decode them in are: " + bob_bases[:10] + "...")

def measure(qubits, bases):
    """This function measures each qubit in the corresponding basis chosen for it."""
    bits = "" # The results of measurements
    for qubit, basis in zip(qubits, bases):
        # Add measurement depending on basis

        if basis == "Z":
            qubit.measure(0, 0)
        elif basis == "X":
            qubit.h(0)
            qubit.measure(0, 0)
        # Execute on Simulator
        simulator = Aer.get_backend('qasm_simulator')
        result = simulator.run(qubit,shots=1).result()
        counts = result.get_counts()
        measured_bit = max(counts, key=counts.get) # Max doesn't matter for simulator   since there is only one shot.
        bits += measured_bit
    return bits

qubits_received = QUANTUM_CHANNEL # Receive qubits from quantum channel
bob_bits = measure(qubits_received, bob_bases)
print("The first few bits Bob received are: " + bob_bits[:10] + "...")

The bases Bob is going to decode them in are: XZZXZZZXXX...
The first few bits Bob received are: 0110111100...


In [8]:
CLASSICAL_CHANNEL = alice_bases # Alice tells Bob which bases she used
# Store the indices of the bases they share in common
common_bases = [i for i in range(KEY_LENGTH) if
CLASSICAL_CHANNEL[i]==bob_bases[i]]
print("The indices of the first 10 bases they share in common are: " +
str(common_bases[:10]))
bob_bits = [bob_bits[index] for index in common_bases]
CLASSICAL_CHANNEL = common_bases # Bob tells Alice which bases they shared in common
alice_bits = [alice_bits[index] for index in common_bases] # Alice keeps only the bits they shared in common

CLASSICAL_CHANNEL = alice_bits[:100] # Alice tells Bob the first 100 bits she has left.
# Bob checks if they match the first 100 bits that he has
if CLASSICAL_CHANNEL == bob_bits[:100]:
    print("Yep, Alice and Bob seem to have the same bits!")
else:
    print("Uh oh, at least one of the bits is different.")

The indices of the first 10 bases they share in common are: [4, 5, 6, 7, 9, 10, 11, 19, 22, 24]
Yep, Alice and Bob seem to have the same bits!


In [9]:
alice_bits = alice_bits[100:] # Alice discards the first 100 bits
bob_bits = bob_bits[100:] # Alice discards the first 100 bits
key = ""
for bit in alice_bits: # Or bob_bits, since both should be the same
    key += bit
print("Shhhhh, the key is:")
print(str(key))
print("Don't tell anyone!")
print("\nThe key is " + str(len(key)) + " bits long.")

Shhhhh, the key is:
010101101001011110110100011011001100000111010101000101011101001000110101100111101001101010000111000111000011001110100100000
Don't tell anyone!

The key is 123 bits long.


In [10]:
# Generating a random string of bits
alice_bits = ""
for i in range(KEY_LENGTH):
    randBit = random.randint(0, 1) # Flip Coin
    alice_bits += str(randBit) # Add randomly chosen bit to the bit string.
# Alice randomly chooses a basis for each bit.

alice_bases = generate_random_bases(KEY_LENGTH)
# Encode Alice's bits
encoded_qubits = encode(alice_bits, alice_bases)
QUANTUM_CHANNEL = encoded_qubits
qubits_intercepted = QUANTUM_CHANNEL # Intercept qubits
eve_bases = generate_random_bases(KEY_LENGTH) # Generate a random set of 
eve_bits = measure(qubits_intercepted, eve_bases) # Measure the qubits
# Eve encodes her decoy qubits and sends them along the quantum channel
QUANTUM_CHANNEL = encode(eve_bits, eve_bases)
bob_bases = generate_random_bases(KEY_LENGTH) # Bob randomly chooses a basis for each bit.
qubits_received = QUANTUM_CHANNEL # Receive qubits from quantum channel
bob_bits = measure(qubits_received, bob_bases)
CLASSICAL_CHANNEL = alice_bases # Alice tells Bob which bases she used
# Store the indices of the bases they share in common
common_bases = [i for i in range(KEY_LENGTH) if
CLASSICAL_CHANNEL[i]==bob_bases[i]]
bob_bits = [bob_bits[index] for index in common_bases]

CLASSICAL_CHANNEL = common_bases # Bob tells Alice which bases they shared in common
alice_bits = [alice_bits[index] for index in common_bases] # Alice keeps only the bits they shared in common
CLASSICAL_CHANNEL = alice_bits[:100] # Alice tells Bob the first 100 bits she has left.
# Bob checks if they match the first 100 bits that he has
if CLASSICAL_CHANNEL == bob_bits[:100]:
    print("Yep, Alice and Bob seem to have the same bits!")
else:
    print("Uh oh, at least one of the bits is different.")

Uh oh, at least one of the bits is different.


# IBM QPU

In [ ]:
QUANTUM_CHANNEL = encoded_qubits
bob_bases = generate_random_bases(KEY_LENGTH) # Bob randomly chooses a basis for each bit.
print("The bases Bob is going to decode them in are: " + bob_bases[:10] + "...")

from qiskit_ibm_provider import IBMProvider
IBMProvider.save_account(token, overwrite=True)
provider = IBMProvider()
list = provider.backends()
l = service.least_busy()
backend = provider.get_backend(l.name)

print("backends available: ", list)
def measure(qubits, bases):
    """This function measures each qubit in the corresponding basis chosen for it."""
    bits = "" # The results of measurements
    for qubit, basis in zip(qubits, bases):
        # Add measurement depending on basis

        if basis == "Z":
            qubit.measure(0, 0)
        elif basis == "X":
            qubit.h(0)
            qubit.measure(0, 0)
        # Execute on Simulator
        qubit = transpile(qubits,backend,optimization_level=3)
        job = backend.run(qubit,shots = 1)
        result = job.result()
        print(result)
    #     counts = result.get_counts()
    #     measured_bit = max(counts, key=counts.get) # Max doesn't matter for simulator   since there is only one shot.
    #     bits += measured_bit
    # return bits
    return result
qubits_received = QUANTUM_CHANNEL # Receive qubits from quantum channel
# bob_bits = measure(qubits_received, bob_bases)
result = measure(qubits_received, bob_bases)
# print("The first few bits Bob received are: " + bob_bits[:10] + "...")

In [ ]:
CLASSICAL_CHANNEL = alice_bases # Alice tells Bob which bases she used
# Store the indices of the bases they share in common
common_bases = [i for i in range(KEY_LENGTH) if
CLASSICAL_CHANNEL[i]==bob_bases[i]]
print("The indices of the first 10 bases they share in common are: " +
str(common_bases[:10]))
bob_bits = [bob_bits[index] for index in common_bases]
CLASSICAL_CHANNEL = common_bases # Bob tells Alice which bases they shared in common
alice_bits = [alice_bits[index] for index in common_bases] # Alice keeps only the bits they shared in common

CLASSICAL_CHANNEL = alice_bits[:100] # Alice tells Bob the first 100 bits she has left.
# Bob checks if they match the first 100 bits that he has
if CLASSICAL_CHANNEL == bob_bits[:100]:
    print("Yep, Alice and Bob seem to have the same bits!")
else:
    print("Uh oh, at least one of the bits is different.")

In [ ]:
alice_bits = alice_bits[100:] # Alice discards the first 100 bits
bob_bits = bob_bits[100:] # Alice discards the first 100 bits
key = ""
for bit in alice_bits: # Or bob_bits, since both should be the same
    key += bit
print("Shhhhh, the key is:")
print(str(key))
print("Don't tell anyone!")
print("\nThe key is " + str(len(key)) + " bits long.")

In [ ]:
# Generating a random string of bits
alice_bits = ""
for i in range(KEY_LENGTH):
    randBit = random.randint(0, 1) # Flip Coin
    alice_bits += str(randBit) # Add randomly chosen bit to the bit string.
# Alice randomly chooses a basis for each bit.

alice_bases = generate_random_bases(KEY_LENGTH)
# Encode Alice's bits
encoded_qubits = encode(alice_bits, alice_bases)
QUANTUM_CHANNEL = encoded_qubits
qubits_intercepted = QUANTUM_CHANNEL # Intercept qubits
eve_bases = generate_random_bases(KEY_LENGTH) # Generate a random set of 
eve_bits = measure(qubits_intercepted, eve_bases) # Measure the qubits
# Eve encodes her decoy qubits and sends them along the quantum channel
QUANTUM_CHANNEL = encode(eve_bits, eve_bases)
bob_bases = generate_random_bases(KEY_LENGTH) # Bob randomly chooses a basis for each bit.
qubits_received = QUANTUM_CHANNEL # Receive qubits from quantum channel
bob_bits = measure(qubits_received, bob_bases)
CLASSICAL_CHANNEL = alice_bases # Alice tells Bob which bases she used
# Store the indices of the bases they share in common
common_bases = [i for i in range(KEY_LENGTH) if
CLASSICAL_CHANNEL[i]==bob_bases[i]]
bob_bits = [bob_bits[index] for index in common_bases]

CLASSICAL_CHANNEL = common_bases # Bob tells Alice which bases they shared in common
alice_bits = [alice_bits[index] for index in common_bases] # Alice keeps only the bits they shared in common
CLASSICAL_CHANNEL = alice_bits[:100] # Alice tells Bob the first 100 bits she has left.
# Bob checks if they match the first 100 bits that he has
if CLASSICAL_CHANNEL == bob_bits[:100]:
    print("Yep, Alice and Bob seem to have the same bits!")
else:
    print("Uh oh, at least one of the bits is different.")